In [141]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels, normalModels

In [142]:
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 20
        self.rounds = 10
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 1.0
        self.drop_rate = 0.0
        self.torch_seed = 0
        self.log_interval = 100
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [143]:
writer = open('20_experiments_fashio.csv', 'a')

In [144]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [145]:
# # Download MNIST manually using 'wget' then uncompress the file
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [146]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

In [147]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
#     client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
#     client['samples'] = len(trainset_ind_list) / args.images
    client['samples'] = len(trainset_ind_list) / (len(trainset_ind_list) * args.clients)

In [148]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = datasets.FashionMNIST('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

In [149]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [150]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
#             data = data.send(client['hook'])
#             target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
            
            if batch_idx % args.log_interval == 0:
#                 loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
#     client['model'].get() 

In [151]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.write('{},'.format(100. * correct / len(test_loader.dataset)))

In [152]:
def Aggregate(all_models, sample_client):
    all_dicts = {}
    for inds, client in enumerate(all_models):
        all_dicts[client['hook'].id] = all_models[inds]['model'].state_dict()
    output_dict = copy.deepcopy(all_dicts[sample_client])
    for key in all_dicts[sample_client]:
        for val in all_dicts:
            if val == sample_client:
                continue
            output_dict[key] += all_dicts[val][key]
#         output_dict[key] /= float(len(all_dicts))
        output_dict[key] /= float(len(all_models))
    output_model = Net().to(device)
    output_model.load_state_dict(output_dict)
    return output_model

In [153]:
torch.manual_seed(args.torch_seed)
global_model = Net()
# torch.manual_seed(args.torch_seed)
# normal_avg = Net()
# torch.manual_seed(args.torch_seed)
# fake_model = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)
#     print('images {} batches {} sample size {}'.format(args.images, args.local_batches, len(client['trainset'])))
#     print(args.local_batches/len(client['trainset']))
    client['pengine'] = PrivacyEngine(
                                       client['model'],
                                       batch_size=args.local_batches,
                                       sample_size=len(client['trainset']),
                                       sample_rate=0.01,
                                       alphas=range(2,32),
                                       noise_multiplier=1.3,
                                       max_grad_norm=1.0,
                                    )
    client['pengine'].attach(client['optim'])

start_time = time.time()

for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
#     normal_avg = normalModels(normal_avg, active_clients)
#     fake_model = Aggregate(active_clients, 'client1')
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
#     test(args, normal_avg, device, global_test_loader, 'Global')
#     test(args, fake_model, device, global_test_loader, 'Global')
    
#     break
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())

writer.write('\n')
end_time = time.time() 
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
writer.write('{:0>2}:{:0>2}:{:05.2f}'.format(int(hours),int(minutes),seconds))
writer.write('\n')
writer.close()

if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

/Users/mohamedghribi/anaconda3/envs/iswc/lib/python3.7/site-packages/opacus/privacy_engine.py:646: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/Users/mohamedghribi/anaconda3/envs/iswc/lib/python3.7/site-packages/opacus/privacy_engine.py:230: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
/Users/mohamedghribi/anaconda3/envs/iswc/lib/python3.7/site-packages/opacus/privacy_engine.py:646: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/Users/mohamedghribi/anaconda3/envs/iswc/lib/python3.7/site-package

Model client16 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.322281
Model client16 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.273688
Model client16 Train Epoch: 3 [0/3008 (0%)]	Loss: 2.242375
Model client16 Train Epoch: 4 [0/3008 (0%)]	Loss: 2.222660
Model client16 Train Epoch: 5 [0/3008 (0%)]	Loss: 2.185804
Model client2 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.314472
Model client2 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.282011
Model client2 Train Epoch: 3 [0/3008 (0%)]	Loss: 2.239011
Model client2 Train Epoch: 4 [0/3008 (0%)]	Loss: 2.205024
Model client2 Train Epoch: 5 [0/3008 (0%)]	Loss: 2.177524
Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.319273
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.288882
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 2.242390
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 2.220141
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 2.183858
Model client5 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.296067
Model client5 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.275640
Mode

Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.949370
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.134197
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.102756
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 2.064229
Model client18 Train Epoch: 4 [0/3008 (0%)]	Loss: 2.054444
Model client18 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.967729
Model client17 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.148718
Model client17 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.122135
Model client17 Train Epoch: 3 [0/3008 (0%)]	Loss: 2.017820
Model client17 Train Epoch: 4 [0/3008 (0%)]	Loss: 2.035613
Model client17 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.915523
Model client19 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.150000
Model client19 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.101135
Model client19 Train Epoch: 3 [0/3008 (0%)]	Loss: 2.073321
Model client19 Train Epoch: 4 [0/3008 (0%)]	Loss: 2.022938
Model client19 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.934688
Model client4 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.13934

Model client4 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.691599
Model client4 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.622909
Model client9 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.860823
Model client9 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.843228
Model client9 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.772507
Model client9 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.728062
Model client9 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.701937
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.866747
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.842972
Model client3 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.776108
Model client3 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.749834
Model client3 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.609555
Model client7 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.861170
Model client7 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.858380
Model client7 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.791669
Model client7 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.688332
Model client7 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.558851
Model client2 

Model client1 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.321691
Model client1 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.317298
Model client1 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.227343
Model client1 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.112004
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.353164
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.373024
Model client3 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.327580
Model client3 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.277810
Model client3 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.267938
Model client14 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.386876
Model client14 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.355590
Model client14 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.316433
Model client14 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.318802
Model client14 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.417944
Model client20 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.365244
Model client20 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.295353
Model client20 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.271136
Model 

Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.120686
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.195891
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.181521
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.083497
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.101180
Model client14 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.069035
Model client14 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.219667
Model client14 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.133862
Model client14 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.176991
Model client14 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.995628
Model client19 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.274153
Model client19 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.226143
Model client19 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.183302
Model client19 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.040641
Model client19 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.069068
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.220736
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.174038

Model client12 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.025669
Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.067204
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.971482
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.141539
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.200370
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.045176

Test set: Average loss for Global model: 1.0263, Accuracy: 6339/10000 (63%)

Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.044670
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.077576
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.132570
Model client18 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.857426
Model client18 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.023310
Model client4 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.963345
Model client4 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.121576
Model client4 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.073211
Model client4 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.090716
Model client4 Train Epoch: 5 [0/3008 (0%)

Model client15 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.840957
Model client15 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.194515
Model client15 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.051831
Model client7 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.124428
Model client7 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.864177
Model client7 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.832196
Model client7 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.002346
Model client7 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.157884
Model client20 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.229861
Model client20 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.855702
Model client20 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.994951
Model client20 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.791373
Model client20 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.937805
Model client12 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.078570
Model client12 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.022577
Model client12 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.995882
Model client12 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.035240
Mo

Model client15 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.928506
Model client15 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.674118
Model client15 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.863098
Model client15 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.211872
Model client1 Train Epoch: 1 [0/3008 (0%)]	Loss: 1.129850
Model client1 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.117574
Model client1 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.939410
Model client1 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.809844
Model client1 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.874025
Model client2 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.964830
Model client2 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.781319
Model client2 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.994360
Model client2 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.037711
Model client2 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.077348
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.690586
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.958289
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.715863
Model c

In [154]:
clients[9]['trainset']